In [ ]:
!pip install PyGithub

In [ ]:
from github import Github
import datetime
import pandas as pd
import numpy as np

In [ ]:
# Replace with your GitHub Personal Access Token
GITHUB_TOKEN = "Added my personal token here"
g = Github(GITHUB_TOKEN)

In [ ]:
def get_repo_data(repo_name):
    repo = g.get_repo(repo_name)
    contributors = repo.get_contributors()
    issues = repo.get_issues(state='all')
    pull_requests = repo.get_pulls(state='all')

    stars = repo.stargazers_count
    forks = repo.forks_count

    contributor_data = []

    for contributor in contributors:
        contributions = repo.get_stats_contributors()

        if contributions:
            for stat in contributions:
                if stat.author.login == contributor.login:
                    contribution_weeks = [week.w for week in stat.weeks if week.c > 0]

                    if contribution_weeks:
                        first_contribution = min(contribution_weeks)
                        last_contribution = max(contribution_weeks)

                        # Fetch other activities
                        contributor_issues = [issue for issue in issues if issue.user.login == contributor.login]
                        contributor_pulls = [pull for pull in pull_requests if pull.user.login == contributor.login]


                       # Convert all activities to datetime objects
                        all_activities = []
                        for week in contribution_weeks:
                            all_activities.append(week)
                        for issue in contributor_issues:
                            all_activities.append(issue.created_at)
                        for pull in contributor_pulls:
                            all_activities.append(pull.created_at)

                        if all_activities:
                            first_activity = min(all_activities)
                            last_activity = max(all_activities)

                            # Calculate Lifespan
                            lifespan = (last_activity - first_activity).days

                        frequency = len(contribution_weeks)
                        retention = (last_contribution - first_contribution) / (
                                    60 * 60 * 24 * 7)  # Convert from seconds to weeks

                        experience = "New" if stat.total < 10 else "Experienced"

                        contributor_data.append({
                            "Contributor": contributor.login,
                            "First Contribution": (first_contribution).replace(tzinfo=None),
                            "Last Contribution": (last_contribution).replace(tzinfo=None),
                            "Total Contributions": stat.total,
                            "Frequency": frequency,
                            "Retention (weeks)": retention,
                            "Experience": experience,
                            "First Activity": first_activity,
                            "Last Activity": last_activity,
                            "Lifespan (days)": lifespan
                        })

    repo_data = {
        "Repository": repo_name,
        "Stars": stars,
        "Forks": forks,
        "Total Issues": issues.totalCount,
        "Total Pull Requests": pull_requests.totalCount,
        "Contributors": contributor_data
    }

    return repo_data


In [ ]:
# Example usage
repo_name = "public-apis/public-apis"  # Updated to target repository
repo_data = get_repo_data(repo_name)

# Convert to DataFrame
columns = ["Contributor", "First Contribution", "Last Contribution", "Total Contributions", "Frequency", "Retention (weeks)", "Experience","First Activity","Last Activity","Lifespan (days)"]
contributors_df = pd.DataFrame(repo_data['Contributors'], columns=columns)
contributors_df = pd.DataFrame(repo_data)
# Data Cleaning: Handling missing values
# contributors_df.fillna({'Total Contributions': 0, 'Frequency': 0, 'Retention (weeks)': 0}, inplace=True)


In [ ]:
# Save to Excel
output_file = "repo_contributor_data_5.xlsx"
contributors_df.to_excel(output_file, index=False)
print(f"Data saved to {output_file}")

Data saved to repo_contributor_data_5.xlsx


# New Section